In [1]:
# Импорт всех необходимых библиотек
!pip install catboost
!pip install xgboost
import numpy as np
import xgboost as xgb
from catboost import CatBoostRegressor
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = pd.read_excel('Данные_для_курсовои_Классическое_МО.xlsx').drop(columns=['Unnamed: 0'])
data.dropna(inplace=True)

# Переместим названия колонок в отдельную переменную, пригодится
columns = data.columns.tolist()

# Быстрая проверка
data.head()

#'IC50, mM', 'CC50, mM', 'SI'

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [2]:
X = data.drop(columns=['IC50, mM','CC50, mM','SI'])
y = data['SI']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
y_pred_lin = linreg.predict(X_test_scaled)

print("Линейная регрессия:")
print("MAE:", mean_absolute_error(y_test, y_pred_lin))
print("MSE:", mean_squared_error(y_test, y_pred_lin))
print("R2:", r2_score(y_test, y_pred_lin))

Линейная регрессия:
MAE: 14418.079515317944
MSE: 40269129769.08358
R2: -19932.991924959246


In [5]:

rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest:")
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("MSE:", mean_squared_error(y_test, y_pred_rf))
print("R2:", r2_score(y_test, y_pred_rf))

Random Forest:
MAE: 201.46691083626945
MSE: 1842787.850511517
R2: 0.08778465434552252


In [6]:
cat_model = CatBoostRegressor(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    loss_function='RMSE',
    random_seed=42,
    verbose=0
)
cat_model.fit(X_train, y_train)

y_pred_cat = cat_model.predict(X_test)

print("CatBoost:")
print("MAE:", mean_absolute_error(y_test, y_pred_cat))
print("MSE:", mean_squared_error(y_test, y_pred_cat))
print("R2:", r2_score(y_test, y_pred_cat))

CatBoost:
MAE: 193.96184552334154
MSE: 1837325.0746910367
R2: 0.09048883319710832


In [7]:
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    random_state=42
)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost:")
print("MAE:", mean_absolute_error(y_test, y_pred_xgb))
print("MSE:", mean_squared_error(y_test, y_pred_xgb))
print("R2:", r2_score(y_test, y_pred_xgb))

XGBoost:
MAE: 192.52831448486745
MSE: 1837311.1452373592
R2: 0.09049572854395882


In [8]:
results_cc50 = pd.DataFrame({
    'Модель': ['Linear Regression', 'Random Forest', 'CatBoost', 'XGBoost'],
    'MAE': [
        mean_absolute_error(y_test, y_pred_lin),
        mean_absolute_error(y_test, y_pred_rf),
        mean_absolute_error(y_test, y_pred_cat),
        mean_absolute_error(y_test, y_pred_xgb)
    ],
    'MSE': [
        mean_squared_error(y_test, y_pred_lin),
        mean_squared_error(y_test, y_pred_rf),
        mean_squared_error(y_test, y_pred_cat),
        mean_squared_error(y_test, y_pred_xgb)
    ],
    'R2': [
        r2_score(y_test, y_pred_lin),
        r2_score(y_test, y_pred_rf),
        r2_score(y_test, y_pred_cat),
        r2_score(y_test, y_pred_xgb)
    ]
})

display(results_cc50)

,Модель,MAE,MSE,R2
0,Linear Regression,14418.079515,4.026913e+10,-19932.991925
1,Random Forest,201.466911,1.842788e+06,0.087785
2,CatBoost,193.961846,1.837325e+06,0.090489
3,XGBoost,192.528314,1.837311e+06,0.090496


В отличие от задач IC50 и CC50, задача регрессии по SI оказалась существенно более сложной. Все используемые нелинейные модели (Random Forest, CatBoost, XGBoost) показывают очень близкие результаты по качеству, однако объясняющая способность остаётся крайне низкой (R² около 9%). Это может быть связано как с высокой сложностью внутренней биохимии показателя SI, так и с отсутствием дополнительных дескрипторов, которые могли бы существенно улучшить предсказание.


---

Линейная регрессия:

MAE 14418 — огромная ошибка.

R² — почти минус 20 000, что говорит о том, что модель даже хуже, чем просто угадывать среднее значение.

Причина в том, что для SI линейная зависимость между признаками и таргетом практически отсутствует — задача сложная, связи очень нелинейные.

---

Random Forest:

После перехода к нелинейной модели ошибка MAE сразу падает примерно до 200 — то есть в 70 раз точнее.

Но R² всё ещё крайне низкий — около 0.09 — то есть объяснено всего 9% дисперсии.


---

CatBoost и XGBoost:

Здесь модели ведут себя практически идентично — обе показывают MAE на уровне ~192-193 и практически одинаковый MSE и R².

Видно, что бустинги дают чуть лучшее качество по сравнению с Random Forest, но прирост минимален.

Это может говорить о том, что сами исходные признаки содержат мало информации для адекватного прогноза SI